In [47]:
import os
import re
import re
import json
import regex
import random
import pprint
import urllib3
import certifi
from time import sleep
from bs4 import BeautifulSoup
from bs4 import NavigableString
from collections import defaultdict

In [2]:
pp = pprint.PrettyPrinter(indent=2)

In [6]:
url = "http://www.theatre-classique.fr/pages/programmes/PageEdition.php"

In [4]:
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

In [7]:
r = http.request('GET', url)

In [8]:
soup = BeautifulSoup(r.data, 'html.parser')

In [81]:
ht_re = re.compile('^HTML$') # find element containing html link

In [14]:
ht = soup(string=ht_re)

In [37]:
tb = soup('tbody')[0]

In [225]:
test = tb.find(string=re.compile('ADENIS'))

In [226]:
test = test.parent.parent.parent.find_all(string=ht_re)[0]

In [229]:
test.parent.parent

<td align="center" width="30"><a href="./edition.php?t=../documents/ADENIS_HOMMEQUINEPEUTPASSIFFLER.xml">HTML</a></td>

In [227]:
test

'HTML'

In [339]:
test = tb(stringa)[random.randint(0, len(tb)-1)]
el = test.parent.parent # local <td> with link
parent = el.parent # main <td> with all categories
print(el)
print('-'*40)
print(parent)
txt_re = re.compile('^TXT$')
result = parent(string=txt_re)

author = parent.find_all(attrs={'href':'../bio/auteurs.htm#'})[0]
print()
print(author.text)
print()
title = author.parent.next_sibling.next_sibling
print()
print(title.text)
print()

# if no txt available, return html
if len(result) == 0: 
    result = el
else: # otherwise return txt el
    result = result[0].parent.parent

print()
print('result:')
print(result)

<td align="center" width="30"><a href="./edition.php?t=../documents/LIQUIER_AMOURELECTRIQUE.xml">HTML</a></td>
----------------------------------------
<tr bgcolor="ivory">
<td><a href="../bio/auteurs.htm#">LIQUIER, Gabriel</a></td> <td>UN AMOUR ÉLECTRIQUE</td>
<td width="40"><a class="date" href="http://fr.wikipedia.org/wiki/1882" target="_new">1882</a></td>
<td width="100">Monologue</td>
<td align="center" width="30"><a href="./edition.php?t=../documents/LIQUIER_AMOURELECTRIQUE.xml">HTML</a></td>
<td align="center" width="40"><a href="../pdf/LIQUIER_AMOURELECTRIQUE.pdf">PDF</a></td><td align="center" width="40"><a href="../documents/LIQUIER_AMOURELECTRIQUE.xml">XML</a></td><td align="center" width="40"><a href="../txt/LIQUIER_AMOURELECTRIQUE.txt">TXT</a></td><td width="40"> </td></tr>

LIQUIER, Gabriel


UN AMOUR ÉLECTRIQUE


result:
<td align="center" width="40"><a href="../txt/LIQUIER_AMOURELECTRIQUE.txt">TXT</a></td>


In [373]:
def get_links(row):
    txt_re = re.compile('^TXT$')
    el = r.parent.parent # local <td> with link
    parent = el.parent # main <td> with all categories
    author = parent.find_all(attrs={'href':'../bio/auteurs.htm#'})[0]
    title = author.parent.next_sibling.next_sibling
    result = parent(string=txt_re) # search for txt category
    # if no txt available, return html
    if len(result) == 0: 
        result = el
        nature = 'xml'
    else: # otherwise return txt el
        result = result[0].parent.parent
        nature = 'txt'
    link = result.a['href']
    return author.text, { 'title': title.text, 
                          'link': link, 
                          'nature': nature }

In [374]:
rows = tb(string=ht_re)
links = defaultdict(list)
for r in rows:
    author, info = get_links(rows)
    links[author].append(info)
pp.pprint(links)

defaultdict(<class 'list'>,
            { 'ABEILLE, Gaspard': [ { 'link': '../txt/ABEILLE_ARGELIE.txt',
                                      'nature': 'txt',
                                      'title': 'ARGÉLIE, REINE DE THESSALIE'},
                                    { 'link': '../txt/ABEILLE_CORIOLAN.txt',
                                      'nature': 'txt',
                                      'title': 'CORIOLAN'},
                                    { 'link': '../txt/ABEILLE_LYNCEE.txt',
                                      'nature': 'txt',
                                      'title': 'LYNCÉE'}],
              'ADENIS, Eugène': [ { 'link': './edition.php?t=../documents/ADENIS_HOMMEQUINEPEUTPASSIFFLER.xml',
                                    'nature': 'xml',
                                    'title': "L'HOMME QUI NE PEUT PAS "
                                             'SIFFLER'}],
              "AIGUEBERRE, Jean Dumas d'": [ { 'link': '../txt/AIGUEBERRE_AVAREAMOUREU

---

## Get Ebook

In [377]:
def get_ebook(author, info):
    url = 'http://www.theatre-classique.fr/pages/programmes/'
    ebook_url = url+info['link']
    r = http.request('GET', ebook_url)    
    if info['nature'] == 'txt':
        return r.data
    else:
        return r
#         ebook = strip_html(ebook)

In [274]:
result.a['href'][3:]

'txt/DONNEAUDEVISE_GENTILHOMMEGUEPIN.txt'

In [555]:
ebook_url = url_up_one_dir + result.a['href'][3:]
ebook_url

'http://www.theatre-classique.fr/pages/txt/LIQUIER_AMOURELECTRIQUE.txt'

In [556]:
r2 = http.request('GET', ebook_url)

In [557]:
ebook = r2.data

In [558]:
ind = ebook.decode('latin-1').find('*\n\n')
ind

953

In [559]:
print(ebook[ind+1:ind+1000].decode('latin-1').strip())

UN AMOUR ÉLECTRIQUE
COMÉDIE EN UN ACTE
THÉÂTRE DE CAMPAGNE - HUITIÈME SÉRIE
1882. Tous droits réservés.
Par M. GABRIEL LIQUIER
PARIS L. FRINZINE, ÉDITEUR  112, Boulevard Saint-Germain, 112



PERSONNAGES.
ARSÈNE GINGINET .......... M. F. GALIPAUX, du Palais-Royal.
CÉLESTE JOLIVAL .......... Mlle ROSAMOND, de la Comédie-Française.
1637.En province. - Un petit salon. - Porte au fond et porte à droite. - Fenêtre à gauche. - Une table avec tiroir, sur laquelle est une sonnette.

Texte issu du Théâtre de Campagne, huitième série. pp. 13-36


			UN AMOUR ÉLECTRIQUE


SCÈNE PREMIÈRE.
Au lever du rideau, la scène est déserte. On entend sonner au dehors. Arsène paraît à droite, une serviette au cou, un rasoir à la main.
ARSÈNE.
Hein ? Vous n'avez pas entendu ?... Il me semble qu'on a sonné à la grille.
		Nouveau coup de sonnette.
Vlan ! Je le disais bien !... Une visite en ce moment... quand je suis sur un cratère... avec mon rasoir anglais !...
		Faisant le geste de se raser.
Allons, bon! J


In [562]:
ind = ebook.decode('latin-1').rfind('\t=')
ind

22930

In [561]:
print(ebook[ind-1000:ind].decode('latin-1').strip())

vraisemblables !... Je vais télégraphier à monsieur votre père.
CÉLESTE.
Il est à Terre-Neuve.
ARSÈNE.
Il y a le câble sous-marin ! Mon amour est capable de traverser l'Océan ! En attendant, vous me permettrez de vous accompagner à Melun ?...
CÉLESTE.
Jamais, monsieur !...
		Changeant de ton.
Du moins, pas encore !...
ARSÈNE.
Mais pour faire ma cour ?
CÉLESTE.
Il y a le télégraphe.
ARSÈNE.
Mais pour nous marier ?... Sera-ce aussi par le télégraphe ?...
CÉLESTE.
Ah ! Dame, alors... nous verrons !
		Lui tendant la main.
Au revoir, monsieur Arsène.
ARSÈNE.
Un instant !.. Je vous suis au bout du monde... à la gare !... Je... 
		Elle s'échappe et lui ferme la porte au nez. 
		Redescendant.
Elle a failli me casser le nez ! C'est un ange!... Je suis le prétendu d'un ange... qui n'est pas sourd ! Ô félicité !... Je cours lui faire un quatrain... de quatre vers !... Non, ce n'est pas assez !... D'une foule de vers !...
		En sortant par la droite.
Et voilà ce que c'est qu'un amour électrique !


In [554]:
print(ebook[ind-1000:ind].decode('latin-1').strip())

---

## HTML-only case

In [376]:
test_auth = 'ADENIS, Eugène'
links[test_auth]

[{'title': "L'HOMME QUI NE PEUT PAS SIFFLER",
  'link': './edition.php?t=../documents/ADENIS_HOMMEQUINEPEUTPASSIFFLER.xml',
  'nature': 'xml'}]

In [379]:
test_ebook = get_ebook(test_auth, links[test_auth][0])

In [382]:
test_soup = BeautifulSoup(test_ebook.data, 'html.parser')

In [603]:
output = ''
blacklist = {
    '[document]',
    'html',
#     'noscript',
#     'header',
    'meta',
    'link',
    'body',
    'head', 
    'div',
    'h2'
#     'input',
    'script',
}

In [425]:
test_soup.body

<body>
<div id="header" onclick="location.href='../../index.html'"></div>
<div id="menu">
<iframe frameborder="0" height="40" marginwidth="0" scrolling="no" src="../menu_2013.html" width="100%"></iframe>
</div><div id="article"><div id="texteTheatre"><p class="titre"><a name="haut"></a>L'HOMME QUI NE PEUT PAS SIFFLER</p><p class="soustitre">MONOLOGUE</p><p class="soustitre">DIT PAR COQUELIN AINÉ, de la Comédie-Française.</p><p class="soustitre">A mon ami GRENET-DANCOURT</p><p class="datedoc">1885. Tous droits réservés.</p>
<p class="auteur">par Eugène Adenis</p>
<p class="imprimeur">PARIS, PAUL OLLENDORFF, ÉDITEUR 28 bis, RUE DE RICHELIEU, 28 bis</p><p class="imprimeur">Imprimerie générale de Châtillon-sur-Seine. A. PICHAT .</p><hr/><p>Texte établi par Paul FIEVRE, août 2019</p>
<p>Publié par Paul Fièvre © Théâtre classique - Version du texte du 30/12/2019 à 13:36:12.</p><hr/><p class="casting"><b>PERSONNAGES</b></p><p class="distribution">UN HOMME.</p><p class="scenographie"></p>
<hr/

In [605]:
all_text = test_soup(text=True)
set([t.parent.name for t in all_text])

{'[document]',
 'a',
 'b',
 'body',
 'div',
 'h2',
 'head',
 'html',
 'link',
 'meta',
 'p',
 'script',
 'small',
 'span',
 'title'}

In [604]:
# i = 0
for a in all_text:
    if a.parent.name not in blacklist:
#     if a == '\n':
#         print(a)
#     elif a.parent.name in {'p', 'b'}:
#         i+=1

#         if 'class' in a.parent.attrs:
#             if a.parent.attrs['class'][0] in ['didascalie', 'didascalieLongue']:
#                 print(a.parent)
#                 print(a)
             
        if a.parent.name == '':
            print(a.parent)
#         print(a.parent.name)
#         print(a.parent)
#         print(a.parent.attrs)
#         print(a)
#         print('-'*40)
#         if i > 5: break

## Other test

In [478]:
xml_only = defaultdict(list)
for author, info in links.items():
    for i in info:
        if i['nature'] == 'xml':
            xml_only[author].append(i)
xml_only

defaultdict(list,
            {'ADENIS, Eugène': [{'title': "L'HOMME QUI NE PEUT PAS SIFFLER",
               'link': './edition.php?t=../documents/ADENIS_HOMMEQUINEPEUTPASSIFFLER.xml',
               'nature': 'xml'}],
             'ALLAIS, Alphone': [{'title': 'LE PAUVRE BOUGRE ET LE BON GÉNIE',
               'link': './edition.php?t=../documents/ALLAIS_BONBOUGRE.xml',
               'nature': 'xml'},
              {'title': 'UN MÉCONTENT',
               'link': './edition.php?t=../documents/ALLAIS_MECONTENT.xml',
               'nature': 'xml'}],
             'BECQUE, Henry': [{'title': 'DOMINO À QUATRE',
               'link': './edition.php?t=../documents/BECQUE_DOMINOAQUATRE.xml',
               'nature': 'xml'},
              {'title': 'UNE EXÉCUTION',
               'link': './edition.php?t=../documents/BECQUE_EXECUTION.xml',
               'nature': 'xml'},
              {'title': 'LA MÈRE',
               'link': './edition.php?t=../documents/BECQUE_MERE.xml',
             

In [505]:
ind = 0
test_auth = 'BECQUE, Henry'
links[test_auth][ind]

{'title': 'DOMINO À QUATRE',
 'link': './edition.php?t=../documents/BECQUE_DOMINOAQUATRE.xml',
 'nature': 'xml'}

In [506]:
test_ebook = get_ebook(test_auth, links[test_auth][ind])

In [507]:
test_soup = BeautifulSoup(test_ebook.data, 'html.parser')

In [508]:
test_soup.body

<body>
<div id="header" onclick="location.href='../../index.html'"></div>
<div id="menu">
<iframe frameborder="0" height="40" marginwidth="0" scrolling="no" src="../menu_2013.html" width="100%"></iframe>
</div><div id="article"><div id="texteTheatre"><p class="titre"><a name="haut"></a>DOMINO À QUATRE</p><p class="soustitre">Comédie en un acte et en prose</p><p class="soustitre">Joué sur la scène du théâtre de l'Odéon le 1er juin 1908</p><p class="datedoc">M CM XXIV</p>
<p class="auteur">d'Henry BECQUE.</p>
<p class="imprimeur">PARIS, Les édition G. GRÈS et compagnie, 21 rue d'Hautefeuille, 21.</p><p class="imprimeur">5543. Tours, Imprimerie E. Arrault, et Cie.</p><hr/><p>Texte établi par Paul FIEVRE septembre 2019.</p>
<p>Publié par Paul Fièvre © Théâtre classique - Version du texte du 30/12/2019 à 17:02:28.</p><hr/><p class="casting"><b>PERSONNAGES</b></p><p class="distribution">ALBANÈS.</p><p class="distribution">SAVARY.</p><p class="distribution">BROCHETON.</p><p class="distributio

In [510]:
output = ''
blacklist = {
    '[document]',
    'html',
#     'noscript',
#     'header',
    'meta',
    'link',
    'body',
    'head', 
    'div',
#     'input',
    'script',
}

In [511]:
all_text = test_soup(text=True)
set([t.parent.name for t in all_text])

{'[document]',
 'a',
 'b',
 'body',
 'div',
 'h2',
 'head',
 'html',
 'link',
 'meta',
 'p',
 'script',
 'small',
 'span',
 'title'}

In [580]:
# i = 0
# start = False
for a in all_text:
#     if a.parent.name not in blacklist:
    if start:
        if a == '\n':
            print(a)        
    if a.parent.name in {'p', 'b'}:
        start = True
#         i+=1 
        print(a.parent.name)
        print(a.parent)
        print(a)
#         print('-'*40)
#         if i > 5: break

p
<p class="titre"><a name="haut"></a>DOMINO À QUATRE</p>
DOMINO À QUATRE
p
<p class="soustitre">Comédie en un acte et en prose</p>
Comédie en un acte et en prose
p
<p class="soustitre">Joué sur la scène du théâtre de l'Odéon le 1er juin 1908</p>
Joué sur la scène du théâtre de l'Odéon le 1er juin 1908
p
<p class="datedoc">M CM XXIV</p>
M CM XXIV


p
<p class="auteur">d'Henry BECQUE.</p>
d'Henry BECQUE.


p
<p class="imprimeur">PARIS, Les édition G. GRÈS et compagnie, 21 rue d'Hautefeuille, 21.</p>
PARIS, Les édition G. GRÈS et compagnie, 21 rue d'Hautefeuille, 21.
p
<p class="imprimeur">5543. Tours, Imprimerie E. Arrault, et Cie.</p>
5543. Tours, Imprimerie E. Arrault, et Cie.
p
<p>Texte établi par Paul FIEVRE septembre 2019.</p>
Texte établi par Paul FIEVRE septembre 2019.


p
<p>Publié par Paul Fièvre © Théâtre classique - Version du texte du 30/12/2019 à 17:02:28.</p>
Publié par Paul Fièvre © Théâtre classique - Version du texte du 30/12/2019 à 17:02:28.
b
<b>PERSONNAGES</b>
PERSON

## Html saved on disk

In [2]:
dr = 'théâtre-classique-source'
files_html = [x for x in os.listdir(dr) if '.html' in x]

In [565]:
randf = files_html[random.randint(0,len(files_html)-1)]

In [609]:
all_els = set()
for fname in files_html:
    with open(os.path.join(dr, fname), 'rb') as f:
        d_test = BeautifulSoup(f.read(), 'html.parser') 
    all_text = d_test(text=True)
    all_els.update([t.parent.name for t in all_text])
all_els

{'[document]',
 'a',
 'b',
 'body',
 'div',
 'h2',
 'head',
 'html',
 'i',
 'link',
 'meta',
 'p',
 'script',
 'small',
 'span',
 'table',
 'td',
 'title',
 'tr'}

In [3]:
blacklist = {
    '[document]',
    'html',
    'meta',
    'link',
    'body',
    'head', 
    'div',
    'script',
    'table',
    'tr',
    'span',
    'small',
    'title'
}

In [27]:
it = files_html.__iter__()

In [81]:
# fname = next(it)
fname = 'abeille-coriolan.html'
print(fname)
with open(os.path.join(dr, fname), 'rb') as f:
    d_test = BeautifulSoup(f.read(), 'html.parser')
all_text = d_test(text=True) 

abeille-coriolan.html


In [83]:
blank_line = re.compile('^\s+$')
class_riddance = {
    'didascalie', 
    'didascalieLongue', 
    'scene', 
    'acte',
    'datedoc', 
    'soustitre', 
    'casting', 
    'imprimeur',
    'info',
    'titre',
    'auteur',
    'distribution',
    'scenographie',
    'texte',
    'note'
}
init_space = re.compile('^\s+')
space_riddance = re.compile('(\s\s+|\n$)')
various_riddances = regex.compile('^\s*(Publié par|Texte (établi|extrait)|Tiré d|Extrait d|[Pp]age \d+|((\p{Lu}+) *\p{posix-punct}*)+$)')
incipit = False

for a in all_text:
    # if a == '\n':
    #    print(a)
    if a.parent.name not in blacklist:
        if a.parent('a', attrs={'name': 'fin'}):
            print('<|finderéplique|>')
            break
        if not re.match(blank_line, a):                
            if 'class' in a.parent.attrs:
                # print(a.parent.name)
                # print(a.parent)                
                if a.parent.attrs['class'][0] in class_riddance:
                    continue
                elif a.parent.attrs['class'][0] == 'proseincise':
#                     print(a.parent)
#                     print(a.encode('utf-8'))                    
                    a = re.sub(init_space, '', a)
                    a = re.sub(space_riddance, ' ', a)
#                     print(a.encode('utf-8'))
                    print(a)
                    continue
                elif a.parent.attrs['class'][0] == 'locuteur':
                    if not incipit:
                        incipit = True
                    else:
                        print('<|finderéplique|>')                        
                    print(a)
                    print('<|débutderéplique|>')
                    continue
        
            # riddance of "Publié par..", "Texte établi..", etc., or full caps lines
            if regex.search(various_riddances, a):
                continue
                
            print(a.parent.name)
            print(a.parent)
            print(a)

p
<p class="prose">Monseigneur,</p>
Monseigneur,
p
<p class="prose">Je ne sais de quel côté je dois regarder Coriolan, pour trouver entre Votre Altesse et lui quelque sorte de ressemblance, que je puisse proposer au public, selon la coutume des auteurs, comme le véritable motif du présent que j'ose vous faire. Coriolan fit une cruelle guerre à sa Patrie. Vous, Monseigneur, non seulement vous vous êtes signalé pour la gloire de la vôtre : Mais comme si vous aviez voulu imiter les Sages de l'Antiquité, qui se vantaient d'avoir tout l'Univers pour Patrie ; Vous avez défendu les limites du monde Chrétien avec autant d'ardeur que si vous eussiez gardé les frontières de la France : et n'avez point fait scrupule d'aller prodiguer le Sang des Bourbons pour le salut des Insulaires de Candie.</p>
Je ne sais de quel côté je dois regarder Coriolan, pour trouver entre Votre Altesse et lui quelque sorte de ressemblance, que je puisse proposer au public, selon la coutume des auteurs, comme le véritab

---

## Removing php error in txt files

In [763]:
err_s = f'blah blah blah ***************** Erreur dans l\'interprétation du texte (ligne {random.randint(100,999)}, programme : edition_txt.php)more blah and still more blah'
err_s

"blah blah blah ***************** Erreur dans l'interprétation du texte (ligne 318, programme : edition_txt.php)more blah and still more blah"

In [761]:
err = re.escape("***************** Erreur dans l'interprétation du texte (ligne ")
err += "[0-9]+"
err += re.escape(", programme : edition_txt.php)")
err_re = re.compile(err)
err_re

re.compile(r"\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\ Erreur\ dans\ l'interprétation\ du\ texte\ \(ligne\ [0-9]+,\ programme\ :\ edition_txt\.php\)",
re.UNICODE)

In [764]:
re.sub(err_re, '', err_s)

'blah blah blah more blah and still more blah'